In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans

In [3]:
df = pd.read_pickle('data/merged_final.pkl')

In [ ]:
df.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,...,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_XNA,clusters,SK_ID_PREV_list,NAME_CONTRACT_TYPE_y_list,AMT_ANNUITY_y_list,AMT_APPLICATION_list,AMT_CREDIT_y_list,AMT_DOWN_PAYMENT_list,AMT_GOODS_PRICE_y_list,WEEKDAY_APPR_PROCESS_START_y_list,HOUR_APPR_PROCESS_START_y_list,FLAG_LAST_APPL_PER_CONTRACT_list,NFLAG_LAST_APPL_IN_DAY_list,RATE_DOWN_PAYMENT_list,RATE_INTEREST_PRIMARY_list,RATE_INTEREST_PRIVILEGED_list,NAME_CASH_LOAN_PURPOSE_list,NAME_CONTRACT_STATUS_list,DAYS_DECISION_list,NAME_PAYMENT_TYPE_list,CODE_REJECT_REASON_list,NAME_TYPE_SUITE_y_list,NAME_CLIENT_TYPE_list,NAME_GOODS_CATEGORY_list,NAME_PORTFOLIO_list,NAME_PRODUCT_TYPE_list,CHANNEL_TYPE_list,SELLERPLACE_AREA_list,NAME_SELLER_INDUSTRY_list,CNT_PAYMENT_list,NAME_YIELD_GROUP_list,PRODUCT_COMBINATION_list,DAYS_FIRST_DRAWING_list,DAYS_FIRST_DUE_list,DAYS_LAST_DUE_1ST_VERSION_list,DAYS_LAST_DUE_list,DAYS_TERMINATION_list,NFLAG_INSURED_ON_APPROVAL_list
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,1,0,0.146609,-0.473145,-0.166254,-0.502536,-0.136483,-1.513915,-0.482188,-0.382775,-0.577879,1,1,0,1,1,0,1,2,2,-0.618500,0,0,0,0,0,0,-1.305167,-2.123391,0.237668,2,0.246839,2,-0.167824,0,1,0,0,0,0,...,0,0,0,0,[1038818],[Consumer loans],[9251.775],[179055.0],[179055.0],[0.0],[179055.0],[SATURDAY],[9],[Y],[1],[0.0],[nan],[nan],[XAP],[Approved],[-606],[XNA],[XAP],[nan],[New],[Vehicles],[POS],[XNA],[Stone],[500],[Auto technology],[24.0],[low_normal],[POS other with interest],[365243.0],[-565.0],[125.0],[-25.0],[-17.0],[0.0]
100003,0,0,0.426872,1.789990,0.613693,1.668688,-1.257951,0.163617,-0.478242,-1.082778,-1.788873,1,1,0,1,1,0,2,1,1,-0.312130,0,0,0,0,0,0,0.572179,0.003138,-0.177140,0,-0.171633,0,0.207637,0,1,0,0,0,0,...,0,0,0,1,"[1810518, 2636178, 2396755]","[Cash loans, Consumer loans, Consumer loans]","[98356.995, 64567.665, 6737.31]","[900000.0, 337500.0, 68809.5]","[1035882.0, 348637.5, 68053.5]","[nan, 0.0, 6885.0]","[900000.0, 337500.0, 68809.5]","[FRIDAY, SUNDAY, SATURDAY]","[12, 17, 15]","[Y, Y, Y]","[1, 1, 1]","[nan, 0.0, 0.10006059514256237]","[nan, nan, nan]","[nan, nan, nan]","[XNA, XAP, XAP]","[Approved, Approved, Approved]","[-746, -828, -2341]","[XNA, Cash through the bank, Cash through the ...","[XAP, XAP, XAP]","[Unaccompanied, Family, Family]","[Repeater, Refreshed, Refreshed]","[XNA, Furniture, Consumer Electronics]","[Cash, POS, POS]","[x-sell, XNA, XNA]","[Credit and cash offices, Stone, Country-wide]","[-1, 1400, 200]","[XNA, Furniture, Consumer electronics]","[12.0, 6.0, 12.0]","[low_normal, middle, middle]","[Cash X-Sell: low, POS industry with interest,...","[365243.0, 365243.0, 365243.0]","[-716.0, -797.0, -2310.0]","[-386.0, -647.0, -1980.0]","[-536.0, -647.0, -1980.0]","[-527.0, -639.0, -1976.0]","[1.0, 0.0, 1.0]"
100004,0,0,-0.413918,-1.166187,-1.439252,-1.104957,-0.780923,0.687501,-0.485139,-0.208769,-0.305754,1,1,1,1,1,0,1,2,2,-0.924870,0,0,0,0,0,0,0.225582,1.255170,-0.591947,0,-0.590104,0,0.223588,0,0,0,0,0,0,...,0,0,0,0,[1564014],[Consumer loans],[5357.25],[24282.0],[20106.0],[4860.0],[24282.0],[FRIDAY],[5],[Y],[1],[0.21200760306744432],[nan],[nan],[XAP],[Approved],[-815],[Cash through the bank],[XAP],[Unaccompanied],[New],[Mobile],[POS],[XN

In [ ]:
df.iloc[:,-36:]

,SK_ID_PREV_list,NAME_CONTRACT_TYPE_y_list,AMT_ANNUITY_y_list,AMT_APPLICATION_list,AMT_CREDIT_y_list,AMT_DOWN_PAYMENT_list,AMT_GOODS_PRICE_y_list,WEEKDAY_APPR_PROCESS_START_y_list,HOUR_APPR_PROCESS_START_y_list,FLAG_LAST_APPL_PER_CONTRACT_list,NFLAG_LAST_APPL_IN_DAY_list,RATE_DOWN_PAYMENT_list,RATE_INTEREST_PRIMARY_list,RATE_INTEREST_PRIVILEGED_list,NAME_CASH_LOAN_PURPOSE_list,NAME_CONTRACT_STATUS_list,DAYS_DECISION_list,NAME_PAYMENT_TYPE_list,CODE_REJECT_REASON_list,NAME_TYPE_SUITE_y_list,NAME_CLIENT_TYPE_list,NAME_GOODS_CATEGORY_list,NAME_PORTFOLIO_list,NAME_PRODUCT_TYPE_list,CHANNEL_TYPE_list,SELLERPLACE_AREA_list,NAME_SELLER_INDUSTRY_list,CNT_PAYMENT_list,NAME_YIELD_GROUP_list,PRODUCT_COMBINATION_list,DAYS_FIRST_DRAWING_list,DAYS_FIRST_DUE_list,DAYS_LAST_DUE_1ST_VERSION_list,DAYS_LAST_DUE_list,DAYS_TERMINATION_list,NFLAG_INSURED_ON_APPROVAL_list
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,[1038818],[Consumer loans],[9251.775],[179055.0],[179055.0],[0.0],[179055.0],[SATURDAY],[9],[Y],[1],[0.0],[nan],[nan],[XAP],[Approved],[-606],[XNA],[XAP],[nan],[New],[Vehicles],[POS],[XNA],[Stone],[500],[Auto technology],[24.0],[low_normal],[POS other with interest],[365243.0],[-565.0],[125.0],[-25.0],[-17.0],[0.0]
100003,"[1810518, 2636178, 2396755]","[Cash loans, Consumer loans, Consumer loans]","[98356.995, 64567.665, 6737.31]","[900000.0, 337500.0, 68809.5]","[1035882.0, 348637.5, 68053.5]","[nan, 0.0, 6885.0]","[900000.0, 337500.0, 68809.5]","[FRIDAY, SUNDAY, SATURDAY]","[12, 17, 15]","[Y, Y, Y]","[1, 1, 1]","[nan, 0.0, 0.10006059514256237]","[nan, nan, nan]","[nan, nan, nan]","[XNA, XAP, XAP]","[Approved, Approved, Approved]","[-746, -828, -2341]","[XNA, Cash through the bank, Cash through the ...","[XAP, XAP, XAP]","[Unaccompanied, Family, Family]","[Repeater, Refreshed, Refreshed]","[XNA, Furniture, Consumer Electronics]","[Cash, POS, POS]","[x-sell, XNA, XNA]","[Credit and cash offices, Stone, Country-wide]","[-1, 1400, 200]","[XNA, Furniture, Consumer electronics]","[12.0, 6.0, 12.0]","[low_normal, middle, middle]","[Cash X-Sell: low, POS industry with interest,...","[365243.0, 365243.0, 365243.0]","[-716.0, -797.0, -2310.0]","[-386.0, -647.0, -1980.0]","[-536.0, -647.0, -1980.0]","[-527.0, -639.0, -1976.0]","[1.0, 0.0, 1.0]"
100004,[1564014],[Consumer loans],[5357.25],[24282.0],[20106.0],[4860.0],[24282.0],[FRIDAY],[5],[Y],[1],[0.21200760306744432],[nan],[nan],[XAP],[Approved],[-815],[Cash through the bank],[XAP],[Unaccompanied],[New],[Mobile],[POS],[XNA],[Regional / Local],[30],[Connectivity],[4.0],[middle],[POS mobile without interest],[365243.0],[-784.0],[-694.0],[-724.0],[-714.0],[0.0]
100006,"[2078043, 2827850, 2190416, 1489396, 1020698, ...","[Cash loans, Revolving loans, Consumer loans, ...","[24246.0, nan, 29027.52, 13500.0, 39954.51, na...","[675000.0, 0.0, 334917.0, 270000.0, 454500.0, ...","[675000.0, 0.0, 267930.0, 270000.0, 481495.5, ...","[nan, nan, 66987.0, nan, nan, nan, 2693.34, na...","[675000.0, nan, 334917.0, 270000.0, 454500.0, ...","[THURSDAY, THURSDAY, SUNDAY, THURSDAY, SATURDA...","[15, 15, 15, 15, 12, 15, 15, 15, 15]","[Y, Y, Y, Y, Y, Y, Y, Y, Y]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[nan, nan, 0.2178298883821148, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[XNA, XAP, XAP, XAP, XNA, XNA, XAP, XNA, XNA]","[Approved, Canceled, Approved, Approved, Appro...","[-181, -181, -311, -181, -438, -181, -617, -18...","[Cash through the bank, XNA, Cash through the ...","[XAP, XAP, XAP, XAP, XAP, XAP, XAP, XAP, LIMIT]","[Unaccompanied, nan, Family, Unaccompanied, na...","[Repeater, Repeater, Repeater, Repeater, Repea...","[XNA, XNA, Audio/Video, XNA, XNA, XNA, Constru...","[Cash, XNA, POS, Cards, Cash, XNA, POS, XNA, C...","[x-sell, XNA, XNA, x-sell, x-sell, XNA, XNA, X...","[Credit and cash offices, Credit and cash offi...","[-1, -1, 8025, -1, -1, -1, 30, -1, -1]","[XNA, XNA, Consumer electronics, XNA, XNA, XNA...","[48.0, nan, 12.0, 0.0, 1

In [4]:
list_columns = df.iloc[:,-36:].columns
list_data = df[list_columns]
df.drop(list_columns,axis=1,inplace=True)

In [ ]:
list_data.head()

,SK_ID_PREV_list,NAME_CONTRACT_TYPE_y_list,AMT_ANNUITY_y_list,AMT_APPLICATION_list,AMT_CREDIT_y_list,AMT_DOWN_PAYMENT_list,AMT_GOODS_PRICE_y_list,WEEKDAY_APPR_PROCESS_START_y_list,HOUR_APPR_PROCESS_START_y_list,FLAG_LAST_APPL_PER_CONTRACT_list,NFLAG_LAST_APPL_IN_DAY_list,RATE_DOWN_PAYMENT_list,RATE_INTEREST_PRIMARY_list,RATE_INTEREST_PRIVILEGED_list,NAME_CASH_LOAN_PURPOSE_list,NAME_CONTRACT_STATUS_list,DAYS_DECISION_list,NAME_PAYMENT_TYPE_list,CODE_REJECT_REASON_list,NAME_TYPE_SUITE_y_list,NAME_CLIENT_TYPE_list,NAME_GOODS_CATEGORY_list,NAME_PORTFOLIO_list,NAME_PRODUCT_TYPE_list,CHANNEL_TYPE_list,SELLERPLACE_AREA_list,NAME_SELLER_INDUSTRY_list,CNT_PAYMENT_list,NAME_YIELD_GROUP_list,PRODUCT_COMBINATION_list,DAYS_FIRST_DRAWING_list,DAYS_FIRST_DUE_list,DAYS_LAST_DUE_1ST_VERSION_list,DAYS_LAST_DUE_list,DAYS_TERMINATION_list,NFLAG_INSURED_ON_APPROVAL_list
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,[1038818],[Consumer loans],[9251.775],[179055.0],[179055.0],[0.0],[179055.0],[SATURDAY],[9],[Y],[1],[0.0],[nan],[nan],[XAP],[Approved],[-606],[XNA],[XAP],[nan],[New],[Vehicles],[POS],[XNA],[Stone],[500],[Auto technology],[24.0],[low_normal],[POS other with interest],[365243.0],[-565.0],[125.0],[-25.0],[-17.0],[0.0]
100003,"[1810518, 2636178, 2396755]","[Cash loans, Consumer loans, Consumer loans]","[98356.995, 64567.665, 6737.31]","[900000.0, 337500.0, 68809.5]","[1035882.0, 348637.5, 68053.5]","[nan, 0.0, 6885.0]","[900000.0, 337500.0, 68809.5]","[FRIDAY, SUNDAY, SATURDAY]","[12, 17, 15]","[Y, Y, Y]","[1, 1, 1]","[nan, 0.0, 0.10006059514256237]","[nan, nan, nan]","[nan, nan, nan]","[XNA, XAP, XAP]","[Approved, Approved, Approved]","[-746, -828, -2341]","[XNA, Cash through the bank, Cash through the ...","[XAP, XAP, XAP]","[Unaccompanied, Family, Family]","[Repeater, Refreshed, Refreshed]","[XNA, Furniture, Consumer Electronics]","[Cash, POS, POS]","[x-sell, XNA, XNA]","[Credit and cash offices, Stone, Country-wide]","[-1, 1400, 200]","[XNA, Furniture, Consumer electronics]","[12.0, 6.0, 12.0]","[low_normal, middle, middle]","[Cash X-Sell: low, POS industry with interest,...","[365243.0, 365243.0, 365243.0]","[-716.0, -797.0, -2310.0]","[-386.0, -647.0, -1980.0]","[-536.0, -647.0, -1980.0]","[-527.0, -639.0, -1976.0]","[1.0, 0.0, 1.0]"
100004,[1564014],[Consumer loans],[5357.25],[24282.0],[20106.0],[4860.0],[24282.0],[FRIDAY],[5],[Y],[1],[0.21200760306744432],[nan],[nan],[XAP],[Approved],[-815],[Cash through the bank],[XAP],[Unaccompanied],[New],[Mobile],[POS],[XNA],[Regional / Local],[30],[Connectivity],[4.0],[middle],[POS mobile without interest],[365243.0],[-784.0],[-694.0],[-724.0],[-714.0],[0.0]
100006,"[2078043, 2827850, 2190416, 1489396, 1020698, ...","[Cash loans, Revolving loans, Consumer loans, ...","[24246.0, nan, 29027.52, 13500.0, 39954.51, na...","[675000.0, 0.0, 334917.0, 270000.0, 454500.0, ...","[675000.0, 0.0, 267930.0, 270000.0, 481495.5, ...","[nan, nan, 66987.0, nan, nan, nan, 2693.34, na...","[675000.0, nan, 334917.0, 270000.0, 454500.0, ...","[THURSDAY, THURSDAY, SUNDAY, THURSDAY, SATURDA...","[15, 15, 15, 15, 12, 15, 15, 15, 15]","[Y, Y, Y, Y, Y, Y, Y, Y, Y]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[nan, nan, 0.2178298883821148, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[XNA, XAP, XAP, XAP, XNA, XNA, XAP, XNA, XNA]","[Approved, Canceled, Approved, Approved, Appro...","[-181, -181, -311, -181, -438, -181, -617, -18...","[Cash through the bank, XNA, Cash through the ...","[XAP, XAP, XAP, XAP, XAP, XAP, XAP, XAP, LIMIT]","[Unaccompanied, nan, Family, Unaccompanied, na...","[Repeater, Repeater, Repeater, Repeater, Repea...","[XNA, XNA, Audio/Video, XNA, XNA, XNA, Constru...","[Cash, XNA, POS, Cards, Cash, XNA, POS, XNA, C...","[x-sell, XNA, XNA, x-sell, x-sell, XNA, XNA, X...","[Credit and cash offices, Credit and cash offi...","[-1, -1, 8025, -1, -1, -1, 30, -1, -1]","[XNA, XNA, Consumer electronics, XNA, XNA, XNA...","[48.0, nan, 12.0, 0.0, 1

In [5]:
for col in list_data.columns:
     list_data[col] = list_data[col].apply(lambda x:x[0])
 #   list_data[col+'prev1'] = list_data[col].apply(lambda x:x[0])
#    list_data[col+'prev2'] = list_data[col].apply(lambda x:x[1])

In [6]:
type(list_data['SK_ID_PREV_list'].iloc[1])

numpy.int64

In [ ]:
list_data

,SK_ID_PREV_list,NAME_CONTRACT_TYPE_y_list,AMT_ANNUITY_y_list,AMT_APPLICATION_list,AMT_CREDIT_y_list,AMT_DOWN_PAYMENT_list,AMT_GOODS_PRICE_y_list,WEEKDAY_APPR_PROCESS_START_y_list,HOUR_APPR_PROCESS_START_y_list,FLAG_LAST_APPL_PER_CONTRACT_list,NFLAG_LAST_APPL_IN_DAY_list,RATE_DOWN_PAYMENT_list,RATE_INTEREST_PRIMARY_list,RATE_INTEREST_PRIVILEGED_list,NAME_CASH_LOAN_PURPOSE_list,NAME_CONTRACT_STATUS_list,DAYS_DECISION_list,NAME_PAYMENT_TYPE_list,CODE_REJECT_REASON_list,NAME_TYPE_SUITE_y_list,NAME_CLIENT_TYPE_list,NAME_GOODS_CATEGORY_list,NAME_PORTFOLIO_list,NAME_PRODUCT_TYPE_list,CHANNEL_TYPE_list,SELLERPLACE_AREA_list,NAME_SELLER_INDUSTRY_list,CNT_PAYMENT_list,NAME_YIELD_GROUP_list,PRODUCT_COMBINATION_list,DAYS_FIRST_DRAWING_list,DAYS_FIRST_DUE_list,DAYS_LAST_DUE_1ST_VERSION_list,DAYS_LAST_DUE_list,DAYS_TERMINATION_list,NFLAG_INSURED_ON_APPROVAL_list
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,1038818,Consumer loans,9251.775,179055.0,179055.0,0.0,179055.0,SATURDAY,9,Y,1,0.000000,NaN,NaN,XAP,Approved,-606,XNA,XAP,NaN,New,Vehicles,POS,XNA,Stone,500,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0
100003,1810518,Cash loans,98356.995,900000.0,1035882.0,NaN,900000.0,FRIDAY,12,Y,1,NaN,NaN,NaN,XNA,Approved,-746,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
100004,1564014,Consumer loans,5357.250,24282.0,20106.0,4860.0,24282.0,FRIDAY,5,Y,1,0.212008,NaN,NaN,XAP,Approved,-815,Cash through the bank,XAP,Unaccompanied,New,Mobile,POS,XNA,Regional / Local,30,Connectivity,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0
100006,2078043,Cash loans,24246.000,675000.0,675000.0,NaN,675000.0,THURSDAY,15,Y,1,NaN,NaN,NaN,XNA,Approved,-181,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,48.0,low_normal,Cash X-Sell: low,365243.0,-151.0,1259.0,-151.0,-143.0,0.0
100007,2001242,Cash loans,16509.600,180000.0,180000.0,NaN,180000.0,SUNDAY,14,Y,1,NaN,NaN,NaN,XNA,Approved,-865,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,Cash,walk-in,Regional / Local,1200,Consumer electronics,18.0,high,Cash Street: high,365243.0,-834.0,-324.0,-354.0,-347.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,2248017,Consumer loans,6605.910,40455.0,40455.0,0.0,40455.0,THURSDAY,17,Y,1,0.000000,NaN,NaN,XAP,Approved,-273,Cash through the bank,XAP,NaN,New,Mobile,POS,XNA,Country-wide,30,Connectivity,8.0,high,POS mobile with interest,365243.0,-210.0,0.0,-30.0,-25.0,0.0
456252,1503599,Consumer loans,10074.465,57595.5,56821.5,3456.0,57595.5,WEDNESDAY,10,Y,1,0.062443,NaN,NaN,XAP,Approved,-2497,Cash through the bank,XAP,Family,New,Consumer Electronics,POS,XNA,Country-wide,190,Consumer electronics,6.0,low_normal,POS household without interest,365243.0,-2466.0,-2316.0,-2316.0,-2311.0,1.0
456253,1686207,Consumer loans,5567.715,28912.5,27306.0,2893.5,28912.5,SATURDAY,12,Y,1,0.104349,NaN,NaN,XAP,Approved,-1909,Cash through the bank,XAP,Unaccompanied,Refreshed,Mobile,POS,XNA,Stone,22,Connectivity,6.0,high,POS mobile with interest,365243.0,-1866.0,-1716.0,-1716.0,-1712.0,0.0


In [7]:
null_values=(list_data.isna().sum()/len(list_data)*100).sort_values(ascending=False)
null_values

RATE_INTEREST_PRIMARY_list           99.442381
RATE_INTEREST_PRIVILEGED_list        99.442381
NAME_TYPE_SUITE_y_list               47.792542
RATE_DOWN_PAYMENT_list               38.712112
AMT_DOWN_PAYMENT_list                38.712112
DAYS_TERMINATION_list                26.367655
NFLAG_INSURED_ON_APPROVAL_list       26.367655
DAYS_FIRST_DRAWING_list              26.367655
DAYS_FIRST_DUE_list                  26.367655
DAYS_LAST_DUE_1ST_VERSION_list       26.367655
DAYS_LAST_DUE_list                   26.367655
AMT_GOODS_PRICE_y_list               15.644135
AMT_ANNUITY_y_list                   14.705424
CNT_PAYMENT_list                     14.704386
PRODUCT_COMBINATION_list              0.016268
NAME_CONTRACT_TYPE_y_list             0.000000
AMT_APPLICATION_list                  0.000000
WEEKDAY_APPR_PROCESS_START_y_list     0.000000
HOUR_APPR_PROCESS_START_y_list        0.000000
FLAG_LAST_APPL_PER_CONTRACT_list      0.000000
NFLAG_LAST_APPL_IN_DAY_list           0.000000
AMT_CREDIT_y_

In [8]:
above_47 = list(null_values[null_values>47].index)
above_47

['RATE_INTEREST_PRIMARY_list',
 'RATE_INTEREST_PRIVILEGED_list',
 'NAME_TYPE_SUITE_y_list']

 ### drop more than 47

In [9]:
list_data.drop(above_47,axis=1,inplace=True)

In [10]:
list_data.drop('SK_ID_PREV_list',axis=1,inplace=True)

In [11]:
null_values=(list_data.isna().sum()/len(list_data)*100).sort_values(ascending=False)
null_values

RATE_DOWN_PAYMENT_list               38.712112
AMT_DOWN_PAYMENT_list                38.712112
DAYS_FIRST_DRAWING_list              26.367655
DAYS_TERMINATION_list                26.367655
NFLAG_INSURED_ON_APPROVAL_list       26.367655
DAYS_FIRST_DUE_list                  26.367655
DAYS_LAST_DUE_1ST_VERSION_list       26.367655
DAYS_LAST_DUE_list                   26.367655
AMT_GOODS_PRICE_y_list               15.644135
AMT_ANNUITY_y_list                   14.705424
CNT_PAYMENT_list                     14.704386
PRODUCT_COMBINATION_list              0.016268
NAME_CASH_LOAN_PURPOSE_list           0.000000
AMT_CREDIT_y_list                     0.000000
WEEKDAY_APPR_PROCESS_START_y_list     0.000000
HOUR_APPR_PROCESS_START_y_list        0.000000
FLAG_LAST_APPL_PER_CONTRACT_list      0.000000
NFLAG_LAST_APPL_IN_DAY_list           0.000000
AMT_APPLICATION_list                  0.000000
CODE_REJECT_REASON_list               0.000000
NAME_CONTRACT_STATUS_list             0.000000
DAYS_DECISION

In [12]:
to_impute = list(null_values[null_values>0].index)
to_impute

['RATE_DOWN_PAYMENT_list',
 'AMT_DOWN_PAYMENT_list',
 'DAYS_FIRST_DRAWING_list',
 'DAYS_TERMINATION_list',
 'NFLAG_INSURED_ON_APPROVAL_list',
 'DAYS_FIRST_DUE_list',
 'DAYS_LAST_DUE_1ST_VERSION_list',
 'DAYS_LAST_DUE_list',
 'AMT_GOODS_PRICE_y_list',
 'AMT_ANNUITY_y_list',
 'CNT_PAYMENT_list',
 'PRODUCT_COMBINATION_list']

In [13]:
numerical_columns = list_data[to_impute].select_dtypes(exclude=object).columns
categorical_columns = list_data[to_impute].select_dtypes(include=object).columns
categorical_columns

Index(['PRODUCT_COMBINATION_list'], dtype='object')

In [14]:
imputer = SimpleImputer(strategy='mean')
list_data[numerical_columns] = imputer.fit_transform(list_data[numerical_columns])

In [15]:
imputer = SimpleImputer(strategy='most_frequent')
list_data[categorical_columns] = imputer.fit_transform(list_data[categorical_columns])

In [16]:
null_values=(list_data.isna().sum()/len(list_data)*100).sort_values(ascending=False)
null_values

NFLAG_INSURED_ON_APPROVAL_list       0.0
DAYS_TERMINATION_list                0.0
AMT_ANNUITY_y_list                   0.0
AMT_APPLICATION_list                 0.0
AMT_CREDIT_y_list                    0.0
AMT_DOWN_PAYMENT_list                0.0
AMT_GOODS_PRICE_y_list               0.0
WEEKDAY_APPR_PROCESS_START_y_list    0.0
HOUR_APPR_PROCESS_START_y_list       0.0
FLAG_LAST_APPL_PER_CONTRACT_list     0.0
NFLAG_LAST_APPL_IN_DAY_list          0.0
RATE_DOWN_PAYMENT_list               0.0
NAME_CASH_LOAN_PURPOSE_list          0.0
NAME_CONTRACT_STATUS_list            0.0
DAYS_DECISION_list                   0.0
NAME_PAYMENT_TYPE_list               0.0
CODE_REJECT_REASON_list              0.0
NAME_CLIENT_TYPE_list                0.0
NAME_GOODS_CATEGORY_list             0.0
NAME_PORTFOLIO_list                  0.0
NAME_PRODUCT_TYPE_list               0.0
CHANNEL_TYPE_list                    0.0
SELLERPLACE_AREA_list                0.0
NAME_SELLER_INDUSTRY_list            0.0
CNT_PAYMENT_list

In [17]:
num = list_data.select_dtypes(exclude='object').columns
cat = list_data.select_dtypes(include='object').columns
print(num,'\n',cat)

Index(['AMT_ANNUITY_y_list', 'AMT_APPLICATION_list', 'AMT_CREDIT_y_list',
       'AMT_DOWN_PAYMENT_list', 'AMT_GOODS_PRICE_y_list',
       'HOUR_APPR_PROCESS_START_y_list', 'NFLAG_LAST_APPL_IN_DAY_list',
       'RATE_DOWN_PAYMENT_list', 'DAYS_DECISION_list', 'SELLERPLACE_AREA_list',
       'CNT_PAYMENT_list', 'DAYS_FIRST_DRAWING_list', 'DAYS_FIRST_DUE_list',
       'DAYS_LAST_DUE_1ST_VERSION_list', 'DAYS_LAST_DUE_list',
       'DAYS_TERMINATION_list', 'NFLAG_INSURED_ON_APPROVAL_list'],
      dtype='object') 
 Index(['NAME_CONTRACT_TYPE_y_list', 'WEEKDAY_APPR_PROCESS_START_y_list',
       'FLAG_LAST_APPL_PER_CONTRACT_list', 'NAME_CASH_LOAN_PURPOSE_list',
       'NAME_CONTRACT_STATUS_list', 'NAME_PAYMENT_TYPE_list',
       'CODE_REJECT_REASON_list', 'NAME_CLIENT_TYPE_list',
       'NAME_GOODS_CATEGORY_list', 'NAME_PORTFOLIO_list',
       'NAME_PRODUCT_TYPE_list', 'CHANNEL_TYPE_list',
       'NAME_SELLER_INDUSTRY_list', 'NAME_YIELD_GROUP_list',
       'PRODUCT_COMBINATION_list'],
      dt

In [18]:
list_data[num].head()

,AMT_ANNUITY_y_list,AMT_APPLICATION_list,AMT_CREDIT_y_list,AMT_DOWN_PAYMENT_list,AMT_GOODS_PRICE_y_list,HOUR_APPR_PROCESS_START_y_list,NFLAG_LAST_APPL_IN_DAY_list,RATE_DOWN_PAYMENT_list,DAYS_DECISION_list,SELLERPLACE_AREA_list,CNT_PAYMENT_list,DAYS_FIRST_DRAWING_list,DAYS_FIRST_DUE_list,DAYS_LAST_DUE_1ST_VERSION_list,DAYS_LAST_DUE_list,DAYS_TERMINATION_list,NFLAG_INSURED_ON_APPROVAL_list
SK_ID_CURR,,,,,,,,,,,,,,,,,
100002,9251.775,179055.0,179055.0,0.000000,179055.0,9,1,0.000000,-606,500,24.0,365243.0,-565.0,125.0,-25.0,-17.0,0.0
100003,98356.995,900000.0,1035882.0,7178.405503,900000.0,12,1,0.081886,-746,-1,12.0,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
100004,5357.250,24282.0,20106.0,4860.000000,24282.0,5,1,0.212008,-815,30,4.0,365243.0,-784.0,-694.0,-724.0,-714.0,0.0
100006,24246.000,675000.0,675000.0,7178.405503,675000.0,15,1,0.081886,-181,-1,48.0,365243.0,-151.0,1259.0,-151.0,-143.0,0.0
100007,16509.600,180000.0,180000.0,7178.405503,180000.0,14,1,0.081886,-865,1200,18.0,365243.0,-834.0,-324.0,-354.0,-347.0,0.0


### Days

In [19]:
days_columns = ['DAYS_DECISION_list','DAYS_FIRST_DRAWING_list','DAYS_FIRST_DUE_list',
                'DAYS_LAST_DUE_1ST_VERSION_list','DAYS_LAST_DUE_list','DAYS_TERMINATION_list']
list_data[days_columns] = np.abs(list_data[days_columns])
list_data[days_columns].head()

,DAYS_DECISION_list,DAYS_FIRST_DRAWING_list,DAYS_FIRST_DUE_list,DAYS_LAST_DUE_1ST_VERSION_list,DAYS_LAST_DUE_list,DAYS_TERMINATION_list
SK_ID_CURR,,,,,,
100002,606.0,365243.0,565.0,125.0,25.0,17.0
100003,746.0,365243.0,716.0,386.0,536.0,527.0
100004,815.0,365243.0,784.0,694.0,724.0,714.0
100006,181.0,365243.0,151.0,1259.0,151.0,143.0
100007,865.0,365243.0,834.0,324.0,354.0,347.0


## Scaling

In [20]:
scaler = StandardScaler()
list_data[num] = scaler.fit_transform(list_data[num])

## Encoding

In [80]:
list_data[cat].head()

,NAME_CONTRACT_TYPE_y_list,WEEKDAY_APPR_PROCESS_START_y_list,FLAG_LAST_APPL_PER_CONTRACT_list,NAME_CASH_LOAN_PURPOSE_list,NAME_CONTRACT_STATUS_list,NAME_PAYMENT_TYPE_list,CODE_REJECT_REASON_list,NAME_CLIENT_TYPE_list,NAME_GOODS_CATEGORY_list,NAME_PORTFOLIO_list,NAME_PRODUCT_TYPE_list,CHANNEL_TYPE_list,NAME_SELLER_INDUSTRY_list,NAME_YIELD_GROUP_list,PRODUCT_COMBINATION_list
SK_ID_CURR,,,,,,,,,,,,,,,
100002,Consumer loans,SATURDAY,Y,XAP,Approved,XNA,XAP,New,Vehicles,POS,XNA,Stone,Auto technology,low_normal,POS other with interest
100003,Cash loans,FRIDAY,Y,XNA,Approved,XNA,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,XNA,low_normal,Cash X-Sell: low
100004,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,New,Mobile,POS,XNA,Regional / Local,Connectivity,middle,POS mobile without interest
100006,Cash loans,THURSDAY,Y,XNA,Approved,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,XNA,low_normal,Cash X-Sell: low
100007,Cash loans,SUNDAY,Y,XNA,Approved,Cash through the bank,XAP,Repeater,XNA,Cash,walk-in,Regional / Local,Consumer electronics,high,Cash Street: high


### Class reduction

In [21]:
def reduce_classes(columns_list):
    for col in columns_list:
        other_classes = list(list_data[col].value_counts()[5:].index)
        list_data[col] = list_data[col].replace(other_classes,'Other')

reduce_classes(['NAME_CASH_LOAN_PURPOSE_list','CODE_REJECT_REASON_list',
                'NAME_GOODS_CATEGORY_list','NAME_SELLER_INDUSTRY_list','PRODUCT_COMBINATION_list'])

In [22]:
list_data = pd.get_dummies(list_data)

In [23]:
list_data.head()

,AMT_ANNUITY_y_list,AMT_APPLICATION_list,AMT_CREDIT_y_list,AMT_DOWN_PAYMENT_list,AMT_GOODS_PRICE_y_list,HOUR_APPR_PROCESS_START_y_list,NFLAG_LAST_APPL_IN_DAY_list,RATE_DOWN_PAYMENT_list,DAYS_DECISION_list,SELLERPLACE_AREA_list,...,NAME_YIELD_GROUP_list_high,NAME_YIELD_GROUP_list_low_action,NAME_YIELD_GROUP_list_low_normal,NAME_YIELD_GROUP_list_middle,PRODUCT_COMBINATION_list_Cash,PRODUCT_COMBINATION_list_Other,PRODUCT_COMBINATION_list_POS household with interest,PRODUCT_COMBINATION_list_POS household without interest,PRODUCT_COMBINATION_list_POS industry with interest,PRODUCT_COMBINATION_list_POS mobile with interest
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-0.397490,0.106002,0.039409,-4.011152e-01,-0.008223,-1.087213,0.048465,-0.955734,-0.408243,0.004086,...,0,0,1,0,0,1,0,0,0,0
100003,6.875222,3.004857,3.217736,5.082078e-17,2.997345,-0.196262,0.048465,0.000000,-0.230463,-0.029315,...,0,0,1,0,0,1,0,0,0,0
100004,-0.715359,-0.516326,-0.550199,-1.295480e-01,-0.653461,-2.275148,0.048465,1.518728,-0.142843,-0.027248,...,0,0,0,1,0,1,0,0,0,0
100006,0.826329,2.100152,1.879075,5.082078e-17,2.059336,0.694689,0.048465,0.000000,-0.947932,-0.029315,...,0,0,1,0,0,1,0,0,0,0
100007,0.194889,0.109802,0.042914,5.082078e-17,-0.004284,0.397705,0.048465,0.000000,-0.079351,0.050753,...,1,0,0,0,0,1,0,0,0,0


In [24]:
df2 = df.merge(list_data,right_index=True,left_index=True)

In [102]:
df2.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,...,NAME_CLIENT_TYPE_list_XNA,NAME_GOODS_CATEGORY_list_Audio/Video,NAME_GOODS_CATEGORY_list_Computers,NAME_GOODS_CATEGORY_list_Consumer Electronics,NAME_GOODS_CATEGORY_list_Mobile,NAME_GOODS_CATEGORY_list_Other,NAME_GOODS_CATEGORY_list_XNA,NAME_PORTFOLIO_list_Cards,NAME_PORTFOLIO_list_Cars,NAME_PORTFOLIO_list_Cash,NAME_PORTFOLIO_list_POS,NAME_PORTFOLIO_list_XNA,NAME_PRODUCT_TYPE_list_XNA,NAME_PRODUCT_TYPE_list_walk-in,NAME_PRODUCT_TYPE_list_x-sell,CHANNEL_TYPE_list_AP+ (Cash loan),CHANNEL_TYPE_list_Car dealer,CHANNEL_TYPE_list_Channel of corporate sales,CHANNEL_TYPE_list_Contact center,CHANNEL_TYPE_list_Country-wide,CHANNEL_TYPE_list_Credit and cash offices,CHANNEL_TYPE_list_Regional / Local,CHANNEL_TYPE_list_Stone,NAME_SELLER_INDUSTRY_list_Connectivity,NAME_SELLER_INDUSTRY_list_Construction,NAME_SELLER_INDUSTRY_list_Consumer electronics,NAME_SELLER_INDUSTRY_list_Furniture,NAME_SELLER_INDUSTRY_list_Other,NAME_SELLER_INDUSTRY_list_XNA,NAME_YIELD_GROUP_list_XNA,NAME_YIELD_GROUP_list_high,NAME_YIELD_GROUP_list_low_action,NAME_YIELD_GROUP_list_low_normal,NAME_YIELD_GROUP_list_middle,PRODUCT_COMBINATION_list_Cash,PRODUCT_COMBINATION_list_Other,PRODUCT_COMBINATION_list_POS household with interest,PRODUCT_COMBINATION_list_POS household without interest,PRODUCT_COMBINATION_list_POS industry with interest,PRODUCT_COMBINATION_list_POS mobile with interest
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,1,0,0.146609,-0.473145,-0.166254,-0.502536,-0.136483,-1.513915,-0.482188,-0.382775,-0.577879,1,1,0,1,1,0,1,2,2,-0.618500,0,0,0,0,0,0,-1.305167,-2.123391,0.237668,2,0.246839,2,-0.167824,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0
100003,0,0,0.426872,1.789990,0.613693,1.668688,-1.257951,0.163617,-0.478242,-1.082778,-1.788873,1,1,0,1,1,0,2,1,1,-0.312130,0,0,0,0,0,0,0.572179,0.003138,-0.177140,0,-0.171633,0,0.207637,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0
100004,0,0,-0.413918,-1.166187,-1.439252,-1.104957,-0.780923,0.687501,-0.485139,-0.208769,-0.305754,1,1,1,1,1,0,1,2,2,-0.924870,0,0,0,0,0,0,0.225582,1.255170,-0.591947,0,-0.590104,0,0.223588,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
100006,0,0,-0.133655,-0.712790,0.187339,-0.653142,-0.928860,0.678084,-0.464985,1.375764,-0.367992,1,1,0,1,0,0,2,2,2,1.526092,0,0,0,0,0,0,0.719504,0.003138,0.237668,0,0.246839,0,0.466533,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0
100007,0,0,-0.189707,-0.201636,-0.367304,-0.050721,0.588283,0.890991,-0.464993,-0.194269,0.308019,1,1,0,1,0,0,1,2,2,-0.312130,0,0,0,0,1,1,-0.992763,0.003138,-0.591947,0,-0.590104,0,-0.133468,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [25]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,f1_score

In [26]:
X = df2.drop('TARGET',axis=1)
y = df2['TARGET']
y = y.astype('int')

In [52]:
X_train1,X_test,y_train1,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [53]:
X_train,X_val,y_train,y_val = train_test_split(X_train1,y_train1,test_size=0.2,random_state=0)

In [54]:
log = LogisticRegression(random_state=42)

In [55]:
log.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
y_pred = log.predict(X_val)

In [57]:
accuracy_score(y_val,y_pred)

0.9189062229584395

In [58]:
recall_score(y_val,y_pred),f1_score(y_val,y_pred)

(0.014889091461561836, 0.02901124925991711)

## Stratified K Fold

In [149]:
scores = cross_val_score(log,X_train1,y_train1,cv=5,scoring='accuracy')

In [150]:
print(scores)
print(np.mean(scores))

[0.91821396 0.91826341 0.91811506 0.91838702 0.91831083]
0.9182580577587173


In [151]:
y_pred = log.predict(X_test)

In [152]:
accuracy_score(y_test,y_pred)

0.9168829970117568